In [1]:
!pip install txgraffiti

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip -q install "pyknotid==0.5.3" "peewee" "pandas"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.0/610.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00


In [3]:
from pyknotid.catalogue.getdb import download_database

# 0.5.3 signature: no overwrite kwarg
download_database()
print("Database downloaded (or already present).")

244MB [00:01, 205MB/s]                           

Successfully downloaded the new database file. Run pyknotid.catalogue.getdb.clean_old_databases to delete old database versions.
Database downloaded (or already present).


In [5]:
import pandas as pd
import numpy as np
from pyknotid.catalogue.database import Knot

MAX_CROSSINGS = 12
PRIME_ONLY = True

# What we would like to extract (we will only select those that exist in your DB schema)
desired_cols = [
    "min_crossings",
    "determinant",
    "signature",
    "unknotting_number",
    "fibered",
    "vassiliev_2",
    "vassiliev_3",
    "hyperbolic_volume",
    "symmetry",
    "composite",
]

available = set(Knot._meta.fields.keys())
use_cols = [c for c in desired_cols if c in available]
print("Available columns used:", use_cols)

fields = [getattr(Knot, c) for c in use_cols]

q = Knot.select(*fields)
if "min_crossings" in available:
    q = q.where(Knot.min_crossings <= MAX_CROSSINGS)
if PRIME_ONLY and "composite" in available:
    q = q.where(Knot.composite == False)

rows = [r.__data__ for r in q]
df_knots = pd.DataFrame(rows)

# Coerce types where relevant
for c in ["min_crossings", "determinant", "signature", "unknotting_number", "vassiliev_2", "vassiliev_3"]:
    if c in df_knots.columns:
        df_knots[c] = pd.to_numeric(df_knots[c], errors="coerce")

if "hyperbolic_volume" in df_knots.columns:
    df_knots["hyperbolic_volume"] = pd.to_numeric(df_knots["hyperbolic_volume"], errors="coerce")
    df_knots["is_hyperbolic"] = df_knots["hyperbolic_volume"].notna()

if "signature" in df_knots.columns:
    df_knots["abs_signature"] = df_knots["signature"].abs()

print("Shape:", df_knots.shape)
df_knots.head()


Available columns used: ['min_crossings', 'determinant', 'signature', 'unknotting_number', 'fibered', 'vassiliev_2', 'vassiliev_3', 'hyperbolic_volume', 'symmetry', 'composite']
Shape: (2978, 12)


,min_crossings,determinant,signature,unknotting_number,fibered,vassiliev_2,vassiliev_3,hyperbolic_volume,symmetry,composite,is_hyperbolic,abs_signature
0,8,15,-2,1.0,True,0,1,6.78371,reversible,False,True,2
1,8,9,0,1.0,True,2,-2,4.12490,reversible,False,True,0
2,10,49,0,2.0,False,-4,-2,10.39450,reversible,False,True,0
3,10,25,0,1.0,False,2,-1,8.90152,reversible,False,True,0
4,10,11,6,3.0,False,7,17,5.86054,reversible,False,True,6


In [8]:
set(df_knots['symmetry'])

{'',
 None,
 'chiral',
 'fully amphicheiral',
 'negative amphicheiral',
 'reversible'}

In [9]:
df_knots['no_symmetry'] = df_knots['symmetry'].map(lambda x : x == None)
df_knots['chiral_symmetry'] = df_knots['symmetry'].map(lambda x : x == 'chiral')
df_knots['fully_amphicheiral_symmetry'] = df_knots['symmetry'].map(lambda x : x == 'fully amphicheiral')
df_knots['negative_amphicheiral_symmetry'] = df_knots['symmetry'].map(lambda x : x == 'negative amphicheiral')
df_knots['reversible_symmetry'] = df_knots['symmetry'].map(lambda x : x == 'reversible')

In [10]:
df_knots.drop(columns=['symmetry'], inplace=True)

In [20]:
df_knots.drop(columns=['no_symmetry'], inplace=True)

In [24]:
df_knots.drop(columns=['is_hyperbolic', 'hyperbolic_volume'], inplace=True)

In [11]:
df_knots.head()

,min_crossings,determinant,signature,unknotting_number,fibered,vassiliev_2,vassiliev_3,hyperbolic_volume,composite,is_hyperbolic,abs_signature,no_symmetry,chiral_symmetry,fully_amphicheiral_symmetry,negative_amphicheiral_symmetry,reversible_symmetry
0,8,15,-2,1.0,True,0,1,6.78371,False,True,2,False,False,False,False,True
1,8,9,0,1.0,True,2,-2,4.12490,False,True,0,False,False,False,False,True
2,10,49,0,2.0,False,-4,-2,10.39450,False,True,0,False,False,False,False,True
3,10,25,0,1.0,False,2,-1,8.90152,False,True,0,False,False,False,False,True
4,10,11,6,3.0,False,7,17,5.86054,False,True,6,False,False,False,False,True


In [38]:


from txgraffiti.graffiti3.heuristics.morgan import morgan_filter
from txgraffiti.graffiti3.heuristics.dalmatian import dalmatian_filter
from txgraffiti.graffiti3.graffiti3 import Graffiti3, Stage


g3 = Graffiti3(
    df_knots,
    max_boolean_arity=2,
    morgan_filter=morgan_filter,
    dalmatian_filter=dalmatian_filter,
    sophie_cfg=dict(
        eq_tol=1e-4,
        min_target_support=5,
        min_h_support=3,
        max_violations=0,
        min_new_coverage=1,
    ),
)

STAGES = [
    # Stage.CONSTANT,
    # Stage.RATIO,
    # Stage.LP1,
    # Stage.LP2,
    # Stage.LP3,
    # Stage.LP4,
    # Stage.POLY_SINGLE,
    # Stage.MIXED,
    Stage.SQRT,
    Stage.LOG,
    Stage.SQRT_LOG,
    # Stage.GEOM_MEAN,
    # Stage.LOG_SUM,
    # Stage.SQRT_PAIR,
    # Stage.SQRT_SUM,
    # Stage.EXP_EXPONENT,

]

# Target invariants to conjecture on.
TARGETS = [
        "unknotting_number",
        # "determinant",
        # "min_crossings",
        # "signature",
        # "vassiliev_2",
        # "vassiliev_3",
        # "signature",
    ]

# Conjecture on the target invariants using the stages defined above.
result = g3.conjecture(
    targets=TARGETS,
    stages=STAGES,
    include_invariant_products=False,
    include_abs=False,
    include_min_max=True,
    include_log=False,
    enable_sophie=True,
    sophie_stages=STAGES,
    quick=True,
    show=True,
)

Stage breakdown: {'unknotting_number': {'sqrt': {'conjectures': 279, 'sophie': 69}, 'log': {'conjectures': 65, 'sophie': 35}, 'sqrt_log': {'conjectures': 8, 'sophie': 11}}}
Total conjectures: 322
Total Sophie conditions: 97

=== Top conjectures (by touch_count, then support) ===

Conjecture 1. unknotting_number ≥ ((1/2) · abs_signature)   [touches=273, support=2978]

Conjecture 2. (reversible_symmetry) ⇒ unknotting_number ≥ 1   [touches=128, support=565]

Conjecture 3. unknotting_number ≥ ((1/11) · max(min_crossings, vassiliev_2))   [touches=119, support=2978]

Conjecture 4. unknotting_number ≥ (((1/21) · log₂(abs_signature)) + (20/21))   [touches=94, support=2978]

Conjecture 5. (reversible_symmetry) ⇒ unknotting_number ≥ (((1/24) · (abs_signature)²) + (5/6))   [touches=66, support=565]

Conjecture 6. (fibered) ⇒ unknotting_number ≥ ((1/11) · max(min_crossings, vassiliev_3))   [touches=64, support=1247]

Conjecture 7. (reversible_symmetry) ⇒ unknotting_number ≥ ((((5/11) · abs_signatu